#### Just add path to test-dataset in 2nd cell here and run all cells to get AUC and Accuracy metrics in the last cell

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import librosa
import librosa.display
import skimage.io
import os
import cv2
from pydub import AudioSegment
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Conv2D, MaxPool2D, MaxPooling2D, BatchNormalization, Flatten, Dropout, ZeroPadding2D, concatenate, Input
from keras.models import Model, Sequential
from keras import regularizers

In [ ]:
testDB = input("Enter path to dataset: ")

In [ ]:
test_files = os.listdir(testDB)

In [ ]:
wav_files = []
txt_files = []

for trfile in test_files:
    if trfile[-3:] == "txt":
        txt_files.append(trfile)
    elif trfile[-3:] == "wav":
        wav_files.append(trfile)

In [ ]:
breathing_cycles = {}

for txfile in txt_files:
    txpath = os.path.join(testDB, txfile)
    t = open(txpath, 'r')
    content = t.readlines()
    breathing_cycles[txfile[:-4]] = []
    
    for line in content:
        data = line[:-1].split("\t")
        breathing_cycles[txfile[:-4]].append((data[0], data[1]))

In [ ]:
os.mkdir("test_parted_audfiles")

for wfile in wav_files:
    wf_path = os.path.join(testDB, wfile)
    bcs = breathing_cycles[wfile[:-4]]
    i = 0
    
    for bc in bcs:
        i += 1
        start = float(bc[0]) * 1000
        end = float(bc[1]) * 1000
        
        AS = AudioSegment.from_wav(wf_path)
        AS = AS[start: end]
        AS.export("test_parted_audfiles/" + wfile[:-4] + "{}".format(i) + ".wav", format = "wav")

In [ ]:
part_wav_files = os.listdir("test_parted_audfiles")

In [ ]:
extra_feats = {}
cls = {'Ar': 0, 'Tc': 1, 'Al': 2, 'Pl': 3, 'Pr': 4, 'Ll': 5, 'Lr': 6}
re = {'LittC2SE': 0, 'Meditron': 1, 'Litt3200': 2, 'AKGC417L': 3}

for pwfile in part_wav_files:
    pwpath = os.path.join(testDB, pwfile)
    extra_feats[pwfile] = []
    
    extra_feats[pwfile].extend([cls[pwfile[8: 10]], re[pwfile[14: 22]]])

In [ ]:
hard_feats = list(extra_feats.values())

In [ ]:
part_wav_files

In [ ]:
# making the spectrograms of the parted audiofiles
os.mkdir("test_parted_spectrograms")

def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

spectrograms = []
srs = []

for pwfile in part_wav_files:
    pwf_path = os.path.join("test_parted_audfiles", pwfile)
    aud, sr = librosa.load(pwf_path)
    
    S = librosa.feature.melspectrogram(y=aud, sr=sr, n_mels=128, fmax=8000, hop_length=512)
    
    fig, ax = plt.subplots()
    S_dB = librosa.power_to_db(S, ref=np.max)
    S_dB = scale_minmax(S_dB, 0, 255).astype(np.uint8)
    S_dB = np.flip(S_dB, axis = 0)
    S_dB = 255 - S_dB
    S_dB = cv2.resize(S_dB, (128, 128), interpolation = cv2.INTER_CUBIC)

    skimage.io.imsave("test_parted_spectrograms/" + pwfile[:-4] + "spec.png", S_dB)

In [ ]:
pspecs = os.listdir("test_parted_spectrograms")

In [ ]:
X_specs = []

for pspec in pspecs:
    pspec_img = cv2.imread(os.path.join("test_parted_spectrograms", pspec))
    X_specs.append(pspec_img)

In [ ]:
# Normalizing the data
for i in range(len(X_specs)):
    X_specs[i] = X_specs[i] / 255.0
    X_specs[i] = X_specs[i].reshape(X_specs[i].shape[0:3])
    X_specs[i] = np.array(X_specs[i].astype("float32"))

In [ ]:
X_test = list(zip(X_specs, hard_feats))

In [ ]:
test_hard = []
test_x = []

for e in X_test:
    test_hard.append(e[1])
    test_x.append(e[0])
    
test_x = np.array(test_x)
test_hard = np.array(test_hard)

In [ ]:
# Y_test = np.array(Y_labels)

In [ ]:
def conv_block():
    return keras.Sequential([
        ZeroPadding2D(padding = (1, 1)),
        Conv2D(64, 3, padding = "valid", activation = "ReLU"),
        BatchNormalization(axis = -1),
        MaxPool2D(pool_size = 2)
    ])

inputs = Input(shape = (128, 128, 3))
hard_feats = Input(shape = (1, ))
x = conv_block()(inputs)
x = conv_block()(x)
x = conv_block()(x)
x = conv_block()(x)
x = Flatten()(x)
x = Dense(32)(x)
x = concatenate([x, hard_feats])
x = BatchNormalization(axis = -1)(x)
x = Dense(2, activation = "sigmoid")(x)

test_model = Model(inputs = [inputs, hard_feats], outputs = [x])

In [ ]:
optimizer = keras.optimizers.SGD(lr = 0.01)

In [ ]:
test_model.compile(optimizer = optimizer, loss = "BinaryCrossentropy", metrics = ["AUC", "accuracy"])

#### Loading model parameters from model file

In [ ]:
test_model.load_weights("live_model1.pt")

In [ ]:
# evaluating on basis of both accuracy and Area Under ROC Curve

def get_test_metrics():
    obtained_metrics = test_model.evaluate([test_x, test_hard[:, 0]], np.array(Y_test), batch_size = 32, verbose = 0)

    print("Obtained Metrics are:")
    print("BinaryCrossentropy Loss = ", obtained_metrics[0])
    print("Area under ROC curve = ", obtained_metrics[1])
    print("Accuracy = ", obtained_metrics[2] * 100, "%")

In [ ]:
# get_test_metrics([y_test, hard_test_y[:, 0]], Y_test)

In [ ]:
predicts = test_model.predict([test_x, test_hard[:, 0]])

In [ ]:
# applied threshold of 0.5 for prediction
predicts[predicts < 0.5] = 0
predicts[predicts >= 0.5] = 1

In [ ]:
print(predicts)